# SELENIUM INSTALLATION

In [ ]:
"""
This is the Selenium installation
"""

!pip install selenium

!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin
!pip install webdriver-manager
"""
Just breaking it up
"""
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

"""
More additions for Selenium
"""

options = webdriver.ChromeOptions()
options.add_argument('-headless') # since we run selenium on Google Colab so we don't want a chrome browser opens, so it will run in the background
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')
options.add_argument("--incognito")
driver = webdriver.Chrome('chromedriver',options=options)
driver.implicitly_wait(30)

"""
I have used his example but just inserted page 1 for our categories Phone - Tablet

"""


tiki_url = 'https://tiki.vn/dien-thoai-may-tinh-bang/c1789?page=1&src=c.1789.hamburger_menu_fly_out_banner'
driver.get(tiki_url)

html_data = driver.page_source   # after driver.get() is done, you can get back HTML string by using .page_source
driver.close() # close the browser after getting what you want
"""
I am pretty sure we now dont need the request.get part any longer so this such be the final set up piece

"""

"""
Below I have added the panda part

"""


Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (89.0.4389.90-0ubuntu0.18.04.2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


'\nBelow I have added the panda part\n\n'

# SCRAPING DATA FROM TIKI - ALL AVAILABLE PAGES OF PHONE AND TABLE CATEGORIES

In [70]:
def tiki_scrape():
  from bs4 import BeautifulSoup
  def selenium_scrap(tiki_url):
    driver = webdriver.Chrome('chromedriver',options=options)        # Define the chrome drivers with setting options we define above  
    driver.implicitly_wait(30)                                       # We let selenium to wait for 30 seconds for all javascript script done before return the result of HTML                       
    driver.get(tiki_url)                                             # Open the browser again to get web page
    html_data = driver.page_source                                   # After driver.get() is done, you can get back HTML string by using .page_source
    driver.close()                                                   # Close the driver after retrieving the web page
                                          
    soup = BeautifulSoup(html_data, 'html.parser')                   # do your beautifulsoup business like the usual
    products = soup.find_all('a', {'class':'product-item'})
    return products

  all_pages =[]
  for i in range (1,1000):
    products = selenium_scrap('https://tiki.vn/dien-thoai-may-tinh-bang/c1789?page='+str(i)+'&src=c.1789.hamburger_menu_fly_out_banner')
    if len(products)== 0:
      break
    else:
      for n in range (len(products)):
        item = products[n]
        all_pages.append(item)

  return all_pages

In [ ]:
raw_data = tiki_scrape()

In [ ]:
raw_data[1]

<a class="product-item" data-view-id="product_list_item" data-view-index="1" href="/dien-thoai-oppo-a12-3gb-32gb-hang-chinh-hang-xanh-duong-p53827214.html?src=ss-organic"><span class="style__StyledItem-sc-18svp8n-0 bHrHjw"><div><div class="thumbnail"><div class="badge-top"></div><img alt="Điện Thoại Oppo A12 (3GB/32GB) - Hàng Chính Hãng" src="https://salt.tikicdn.com/cache/280x280/ts/product/aa/f2/95/c0c73883ea383b2eedbeda927ab615dd.jpg"/></div><div class="style__OptionColor-sc-18svp8n-2 FzDve option-color"><div class="Slide__Root-sc-1bhg4sj-0 cIzQMm" style="width: 144px; margin: 16px auto 12px 0px;"><div class="Slide__Slider-sc-1bhg4sj-1 IZyLq"><div class="Slide__Frame-sc-1bhg4sj-2 hwwfcE"><div class="Slide__Item-sc-1bhg4sj-3 hMoQqt" style="border: 1px solid rgb(253, 130, 10);" width="25"><img class="style__BlockImg-sc-18svp8n-5 kmGwFP" src="https://salt.tikicdn.com/cache/64x64/ts/product/aa/f2/95/c0c73883ea383b2eedbeda927ab615dd.jpg"/></div><div class="Slide__Item-sc-1bhg4sj-3 hMoQqt

#EXTRACTING DATA FROM SCRAPING

In [65]:
import re
pattern = r"(p\d+)"
product_Id = re.search("(p\d+)",raw_data[1]['href']).group()
product_Id

"""
def img_url(d): ---------------------------> Idea for the image url.
  img = raw_data[n].find_all("img")
  pattern = r".*/280x280/.*"
  product_img = []
  image_len = len(img)

  for i in range(image_len-1): 
    match_obj = re.findall(pattern,img[n]['src'])
    if match_obj:
      product_img.append(match_obj)
    else:
      return product_img
"""

def refund(a):
  """
  Rẻ hơn hoàn tiền
  To find out product which allows to get refund if customer could find the product cheaper anywhere else. False: Option not applicable
  """
  if len(raw_data[a].find('div',{'class':'badge-under-price'})) == 1: 
      return (True)
  else: 
      return (False)

def is_Now(b):
  """
  To find if the product has "Tiki now" option (to be delivered within the day):
  False: Tiki_now not applicable
  """
  if len(raw_data[b].find('div',{'class':'badge-service'})) == 1:
      return True
  else: 
      return False

def free_gift(c): #Not too sure why this doesnt work
  """
  To find if the product has a free gift icon
  """
  if raw_data[c].find("div",{"class": "freegift-list"}):
    return True
  else: 
    return False        

#def product_id(url): # ignore for now just an idea to get the sku number. The idea is to send another .get request as the sku number is inside another page once you click on the product image.
  """
  To find the sku number for each of our products
  """
  #driver = webdriver.Chrome('chromedriver',options=options)
  #driver.get('tiki.vn'+raw_data[n]['href'])  
  #driver.implicitly_wait(30)
  #b=  driver.page_source
  #sku_soup = BeautifulSoup(b, 'html.parser') 
  #sku = sku_soup.find('div', {'class': 'content has-table'}).text[-13:]
  #return sku              

def data_extract():
  data = []
  for n in range(len(raw_data)):
    d = {
        "product_ID": "",
        "product_title" : "",
        "price": "",
        "discount": "",
        "freeship": "",
        "product_img_url":"",
        "product_pg_url":"",
        "tiki_now":"",
        "installment_accepted":"",
        "total_reviews": "",
        "ratings_avg": "",
        "cheaper_getRefund":"","free_gift": ""
    }
    try:
      d["product_ID"] = ((re.search("(p\d+).html",raw_data[n]['href'])).group()).strip('.html')
      d["product_title"] = raw_data[n].find('div', {'class':'name'}).text
      d["price"] = raw_data[n].find('div', {'class':'price-discount__price'}).text
      if raw_data[n].find('div', {'class':'price-discount__discount'}) is None:
        d["discount"] = ""
      else:
        d["discount"] = raw_data[n].find('div', {'class':'price-discount__discount'}).text
      if raw_data[n].find('div', {'class':'thumbnail'}).text == '':
        d["freeship"] = ''
        d["product_img_url"] = raw_data[n].find('img').get('src')
      else:
        d["freeship"] = raw_data[n].find('div', {'class':'thumbnail'}).text
        d["product_img_url"] = raw_data[n].find_all('img')[1].get('src') #img_url(n)
      d["product_pg_url"] = 'tiki.vn'+raw_data[n]['href']
      d["tiki_now"] = is_Now(n)
      d["installment_accepted"] = raw_data[n].find('div', {'class':'badge-benefits'}).text# Shoots out result in Vietnamese which I think is fine.
      if raw_data[n].find('div', {'class':'review'}) is None:
        d["total_reviews"] = ""
      else:
        d["total_reviews"] = raw_data[n].find('div', {'class':'review'}).text
      if raw_data[n].find('div', {'class':'rating__average'}) is None:
        d["ratings_avg"] = ""
      else:
        d["ratings_avg"] = raw_data[n].find('div', {'class':'rating__average'}).get("style").strip('width:')  
      d["cheaper_getRefund"] = refund(n)
      #d["free_gift"] = free_gift(n)# My attempt at Free Gift output
      data.append(d)
    except:
      print ("Error found!")
  return data

In [66]:
data = data_extract()

In [67]:
data

[{'cheaper_getRefund': False,
  'discount': '-34%',
  'free_gift': '',
  'freeship': '',
  'installment_accepted': '',
  'price': '3.490.000 ₫',
  'product_ID': 'p72042076',
  'product_img_url': 'https://salt.tikicdn.com/cache/280x280/ts/product/83/1d/47/b6ee30f0007f9d8813b8abaa0f101dd8.jpg',
  'product_pg_url': 'tiki.vn//tka.tiki.vn/pixel?data=djAwMRYZr1Sxnk8xZHall_iRiMdPbpuRuJ7w2mvftAYktLv-9IVgWvEsiZyDJbLFPytly80fIqJjLWjga-r2ptDA3pAddlQry5fnWk0K_119gstSWuaXhA2rkJB-CBZSWg77uq6VP6ToK6p_UQKJb-v_Vbdflk8dW6eQ45cvHG1Cq1jnDCQDeEHV76S8IN-JHkOkfDBkdCB91-h-yUKaWBqni2nv4_neMl28eD-h8MEHkqL9CTZ0zeVmxWyYNpTjmgOETCOf8g0z9GvlrShCFYYXsHsDo70IqkFCAWs1ryjnt0QZqcPBh36NgM13-LR7B_E8sB1mVeEtl8dMECUFJl5KmMSU3r4GimgiQqrpXcFIEEG4CdpImJleTZ3RXjDnkvuvsT-Ssrp4AlhWIcBZ2WZipC2vhdqN7W3qzgtW11SHh7z51Mhgjg5QJRotfN5EKBU7ro_zKSTo1mt0hYSJRkrSO86-qGEMlnRNJlc_zz0GR5YknYFTC0g2X_--9SuS2s4pRIZje8W5FyypX-EWMo7BjsGCOH8cJSfAWnvl5v3RdzeMsur8IuD8D7ST7rizwG7nEPA_uDStZIIIVmU-KwFUvTCDuUwEuiqOlKdFXdw3QAVXVziLgZ67aaKAxSHYQCBN8DjGsN_f8

In [68]:
import pandas as pd
products = pd.DataFrame(data = data, columns = data[0].keys())
products

,product_ID,product_title,price,discount,freeship,product_img_url,product_pg_url,tiki_now,installment_accepted,total_reviews,ratings_avg,cheaper_getRefund,free_gift
0,p72042076,AdMáy tính bảng Vankyo MatrixPad S20 - Hàng Ch...,3.490.000 ₫,-34%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn//tka.tiki.vn/pixel?data=djAwMRYZr1Sxnk...,False,,(5),92%;,False,
1,p53827214,Điện Thoại Oppo A12 (3GB/32GB) - Hàng Chính Hãng,2.550.000 ₫,-15%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-oppo-a12-3gb-32gb-hang-chin...,True,,(164),94%;,False,
2,p63839533,Điện Thoại Vsmart Live 4 (6GB/64GB) - Hàng Chí...,3.390.000 ₫,-29%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-vsmart-live-4-hang-chinh-ha...,True,,(858),94%;,True,
3,p58065346,Điện Thoại Realme C11 (2GB/32GB) - Hàng Chính ...,2.190.000 ₫,-19%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-realme-c11-2gb-32gb-hang-ch...,True,,(82),96%;,False,
4,p66773643,AdMáy đọc sách All New Kindle Bản đặc biệt 8GB...,2.332.000 ₫,-27%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn//tka.tiki.vn/pixel?data=djAwMSU6V1ykvY...,False,,(13),90%;,True,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
419,p16518729,Điện thoại để bàn Panasonic KX-TS580 hàng chín...,735.000 ₫,-23%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-de-ban-panasonic-kx-ts580-h...,False,,,,False,
420,p65091422,"Điện thoại Oukitel WP6 (Chống va đập,chống nướ...",6.990.000 ₫,-13%,Freeship,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-oukitel-wp6-chong-va-dap-ch...,False,,,,False,
421,p87054916,HUAWEI P40 Pro+ 5G (8+512G)| Bộ 5 Camera Siêu ...,24.990.000 ₫,-22%,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/huawei-p40-pro-5g-8-512g-bo-5-camera-s...,False,,,,False,
422,p84441714,Điện Thoại Xiaomi Redmi Note 10 (6GB/128GB)- H...,5.490.000 ₫,,,https://salt.tikicdn.com/cache/280x280/ts/prod...,tiki.vn/dien-thoai-xiaomi-redmi-note-10-6gb-12...,False,,,,False,


In [69]:
products.to_pickle("./result.pkl")
unpickled_result = pd.read_pickle("./result.pkl")
unpickled_result.to_csv("./result.csv", index=False)